In [1]:
import numpy as np
import pandas as pd
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.feature_selection import SelectKBest, f_regression, f_classif
import gc
gc.enable()
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=FutureWarning)
from sklearn.preprocessing import StandardScaler, LabelEncoder, MinMaxScaler
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import roc_auc_score, auc, roc_curve
from bayes_opt import BayesianOptimization
import lightgbm as lgb
from sklearn.calibration import calibration_curve,CalibratedClassifierCV
from pycaret.classification import *


file = "dataset/transformed_dataset.csv"
reg_cols = ['atlas_pct_diabetes_adults13',
 'atlas_pct_wic15',
 'total_physician_office_net_paid_pmpm_cost_9to12m_b4',
 'atlas_pct_laccess_hisp15',
 'atlas_pct_fmrkt_frveg16',
 'credit_hh_nonmtgcredit_60dpd',
 'atlas_dirsales_farms12',
 'rx_nonmaint_pmpm_ct',
 'zip_cd',
 'atlas_pct_laccess_white15',
 'credit_hh_bankcard_severederog',
 'atlas_pct_fmrkt_credit16',
 'credit_bal_autofinance_new',
 'rej_days_since_last_clm',
 'rx_generic_pmpm_ct_0to3m_b4',
 'rwjf_social_associate_rate',
 'med_physician_office_ds_clm_6to9m_b4',
 'atlas_totalocchu',
 'atlas_veg_acrespth12',
 'atlas_pct_loclsale12',
 'atlas_pct_fmrkt_anmlprod16',
 'atlas_freshveg_farms12',
 'rwjf_resident_seg_black_inx',
 'atlas_pct_loclfarm12',
 'total_outpatient_mbr_resp_pmpm_cost_6to9m_b4',
 'atlas_berry_acrespth12',
 'rx_maint_pmpm_ct_9to12m_b4',
 'rx_tier_2_pmpm_ct',
 'atlas_agritrsm_rct12',
 'atlas_pct_laccess_snap15',
 'atlas_deep_pov_all',
 'ccsp_227_pct',
 'bh_outpatient_net_paid_pmpm_cost',
 'atlas_veg_farms12',
 'rx_hum_16_pmpm_ct',
 'cms_risk_adjustment_factor_a_amt',
 'atlas_recfac14',
 'total_physician_office_copay_pmpm_cost',
 'atlas_pc_fsrsales12',
 'atlas_pct_fmrkt_baked16',
 'atlas_net_international_migration_rate',
 'rx_maint_mbr_resp_pmpm_cost_6to9m_b4',
 'rx_generic_pmpm_cost_6to9m_b4',
 'rx_gpi2_49_pmpm_cost_0to3m_b4',
 'atlas_pct_sbp15',
 'atlas_pct_laccess_child15',
 'met_obe_diag_pct',
 'atlas_orchard_acrespth12',
 'atlas_pct_laccess_hhnv15',
 'cnt_cp_webstatement_pmpm_ct',
 'atlas_pct_laccess_lowi15',
 'rx_gpi2_02_pmpm_cost',
 'cms_partd_ra_factor_amt',
 'atlas_pct_free_lunch14',
 'rx_tier_2_pmpm_ct_3to6m_b4',
 'cons_chva',
 'atlas_pct_fmrkt_wiccash16',
 'rx_overall_net_paid_pmpm_cost_6to9m_b4',
 'total_med_allowed_pmpm_cost_9to12m_b4',
 'bh_physician_office_copay_pmpm_cost_6to9m_b4',
 'atlas_pct_snap16',
 'atlas_ghveg_sqftpth12',
 'atlas_pc_dirsales12',
 'atlas_pct_reduced_lunch14',
 'ccsp_236_pct',
 'atlas_deep_pov_children',
 'atlas_pct_sfsp15',
 'rwjf_air_pollute_density',
 'rx_generic_pmpm_cost',
 'cms_tot_partd_payment_amt',
 'cons_nwperadult',
 'rx_days_since_last_script',
 'atlas_pct_laccess_nhasian15',
 'rx_nonbh_mbr_resp_pmpm_cost_6to9m_b4',
 'rx_days_since_last_script_6to9m_b4',
 'atlas_pct_obese_adults13',
 'credit_bal_consumerfinance',
 'atlas_pct_fmrkt_wic16',
 'atlas_orchard_farms12',
 'atlas_berry_farms12',
 'atlas_pct_laccess_multir15',
 'rx_bh_mbr_resp_pmpm_cost_9to12m_b4',
 'atlas_pc_wic_redemp12',
 'rwjf_mv_deaths_rate',
 'atlas_povertyunder18pct',
 'rx_gpi2_72_pmpm_cost_6to9m_b4',
 'atlas_pct_fmrkt_snap16',
 'atlas_medhhinc',
 'rx_nonbh_net_paid_pmpm_cost',
 'credit_bal_bankcard_severederog',
 'bh_ip_snf_net_paid_pmpm_cost',
 'atlas_pc_snapben15',
 'rx_nonbh_pmpm_ct_0to3m_b4',
 'rx_overall_mbr_resp_pmpm_cost_0to3m_b4',
 'auth_3mth_post_acute_mean_los',
 'rx_branded_mbr_resp_pmpm_cost',
 'rx_tier_1_pmpm_ct_0to3m_b4',
 'bh_ncdm_pct',
 'atlas_naturalchangerate1016',
 'rx_mail_mbr_resp_pmpm_cost_0to3m_b4',
 'credit_bal_autobank',
 'rx_nonotc_dist_gpi6_pmpm_ct',
 'cons_cgqs',
 'rx_overall_gpi_pmpm_ct_0to3m_b4',
 'credit_hh_bankcardcredit_60dpd',
 'rx_gpi2_01_pmpm_cost_0to3m_b4',
 'cci_dia_m_pmpm_ct',
 'atlas_pct_nslp15',
 'mcc_end_pct',
 'atlas_pct_laccess_black15',
 'credit_bal_mtgcredit_new',
 'credit_hh_1stmtgcredit',
 'cons_chmi',
 'rwjf_income_inequ_ratio',
 'atlas_pct_laccess_pop15',
 'atlas_pc_ffrsales12',
 'atlas_hh65plusalonepct',
 'atlas_pct_fmrkt_sfmnp16',
 'auth_3mth_acute_mean_los',
 'rx_hum_28_pmpm_cost',
 'atlas_pct_laccess_nhna15',
 'atlas_povertyallagespct',
 'rx_nonbh_mbr_resp_pmpm_cost',
 'rx_nonmaint_mbr_resp_pmpm_cost_9to12m_b4',
 'atlas_pct_fmrkt_otherfood16',
 'lab_dist_loinc_pmpm_ct',
 'rx_generic_mbr_resp_pmpm_cost',
 'atlas_pct_laccess_seniors15',
 'atlas_pct_cacfp15',
 'total_outpatient_allowed_pmpm_cost_6to9m_b4',
 'rx_nonmaint_mbr_resp_pmpm_cost',
 'credit_bal_nonmtgcredit_60dpd',
 'atlas_ownhomepct',
 'rx_overall_mbr_resp_pmpm_cost',
 'atlas_redemp_snaps16',
 'atlas_netmigrationrate1016',
 'atlas_percapitainc',
 'phy_em_px_pct',
 'rx_generic_mbr_resp_pmpm_cost_0to3m_b4']

cat_cols = ['bh_ncdm_ind',
 'auth_3mth_post_acute_inf',
 'rx_maint_net_paid_pmpm_cost_t_9-6-3m_b4',
 'ccsp_065_pmpm_ct',
 'auth_3mth_acute_vco',
 'rx_gpi2_72_pmpm_ct_6to9m_b4',
 'auth_3mth_post_acute_men',
 'rej_total_physician_office_visit_ct_pmpm_0to3m_b4',
 'total_physician_office_net_paid_pmpm_cost_t_9-6-3m_b4',
 'bh_ip_snf_net_paid_pmpm_cost_0to3m_b4',
 'mcc_ano_pmpm_ct_t_9-6-3m_b4',
 'atlas_type_2015_update',
 'atlas_retirement_destination_2015_upda',
 'auth_3mth_post_acute_sns',
 'atlas_hiamenity',
 'cons_ltmedicr',
 'auth_3mth_acute_ccs_086',
 'total_physician_office_mbr_resp_pmpm_cost_t_9-6-3m_b4',
 'auth_3mth_acute_cir',
 'atlas_csa12',
 'total_med_net_paid_pmpm_cost_t_6-3-0m_b4',
 'cons_n2pwh',
 'auth_3mth_snf_post_hsp',
 'auth_3mth_post_acute_inj',
 'med_outpatient_mbr_resp_pmpm_cost_t_9-6-3m_b4',
 'rx_gpi2_56_dist_gpi6_pmpm_ct_3to6m_b4',
 'atlas_low_employment_2015_update',
 'auth_3mth_acute_inf',
 'lab_albumin_loinc_pmpm_ct',
 'rx_gpi2_17_pmpm_cost_t_12-9-6m_b4',
 'cons_rxadhs',
 'cons_mobplus',
 'atlas_foodinsec_child_03_11',
 'lang_spoken_cd',
 'bh_ip_snf_mbr_resp_pmpm_cost_9to12m_b4',
 'auth_3mth_post_acute_gus',
 'auth_3mth_acute_cad',
 'rx_maint_pmpm_ct_t_6-3-0m_b4',
 'auth_3mth_acute_ccs_044',
 'cons_hxmioc',
 'med_outpatient_visit_ct_pmpm_t_12-9-6m_b4',
 'med_physician_office_allowed_pmpm_cost_t_9-6-3m_b4',
 'auth_3mth_acute_res',
 'auth_3mth_acute_chf',
 'auth_3mth_acute_ccs_030',
 'auth_3mth_dc_hospice',
 'auth_3mth_acute_neo',
 'atlas_type_2015_recreation_no',
 'hum_region',
 'atlas_ghveg_farms12',
 'rx_maint_net_paid_pmpm_cost_t_12-9-6m_b4',
 'auth_3mth_acute_ccs_048',
 'rx_overall_gpi_pmpm_ct_t_6-3-0m_b4',
 'rx_overall_gpi_pmpm_ct_t_12-9-6m_b4',
 'rx_nonbh_pmpm_ct_t_9-6-3m_b4',
 'mcc_chf_pmpm_ct_t_9-6-3m_b4',
 'auth_3mth_post_acute_chf',
 'auth_3mth_psychic',
 'rx_nonotc_pmpm_cost_t_6-3-0m_b4',
 'auth_3mth_acute_end',
 'atlas_low_education_2015_update',
 'src_div_id',
 'auth_3mth_bh_acute',
 'auth_3mth_acute_ccs_067',
 'atlas_type_2015_mining_no',
 'cons_n2pmr',
 'rx_mail_net_paid_pmpm_cost_t_6-3-0m_b4',
 'rej_med_er_net_paid_pmpm_cost_t_9-6-3m_b4',
 'med_outpatient_deduct_pmpm_cost_t_9-6-3m_b4',
 'rej_med_ip_snf_coins_pmpm_cost_t_9-6-3m_b4',
 'rx_generic_dist_gpi6_pmpm_ct_t_9-6-3m_b4',
 'auth_3mth_dc_home',
 'auth_3mth_acute_bld',
 'auth_3mth_acute_ner',
 'oontwk_mbr_resp_pmpm_cost_t_6-3-0m_b4',
 'rx_gpi2_90_dist_gpi6_pmpm_ct_9to12m_b4',
 'atlas_foodhub16',
 'rx_maint_pmpm_cost_t_6-3-0m_b4',
 'auth_3mth_post_acute_ben',
 'est_age',
 'auth_3mth_post_acute_cer',
 'auth_3mth_acute_ccs_153',
 'auth_3mth_acute_dig',
 'total_ip_maternity_net_paid_pmpm_cost_t_12-9-6m_b4',
 'auth_3mth_post_acute_cad',
 'rx_bh_pmpm_ct_0to3m_b4',
 'rx_nonmail_dist_gpi6_pmpm_ct_t_9-6-3m_b4',
 'atlas_persistentchildpoverty_1980_2011',
 'atlas_slhouse12',
 'atlas_population_loss_2015_update',
 'auth_3mth_acute_ccs_094',
 'auth_3mth_post_acute_ner',
 'auth_3mth_acute_ccs_227',
 'rx_overall_dist_gpi6_pmpm_ct_t_6-3-0m_b4',
 'auth_3mth_acute_trm',
 'auth_3mth_post_acute',
 'auth_3mth_acute_dia',
 'auth_3mth_acute_ccs_043',
 'rx_overall_mbr_resp_pmpm_cost_t_6-3-0m_b4',
 'cms_orig_reas_entitle_cd',
 'auth_3mth_post_acute_end',
 'auth_3mth_acute_can',
 'auth_3mth_acute_ccs_172',
 'auth_3mth_dc_home_health',
 'atlas_hipov_1115',
 'rx_phar_cat_cvs_pmpm_ct_t_9-6-3m_b4',
 'rx_gpi2_62_pmpm_cost_t_9-6-3m_b4',
 'cons_n2phi',
 'auth_3mth_post_acute_hdz',
 'auth_3mth_bh_acute_mean_los',
 'auth_3mth_post_acute_dig',
 'auth_3mth_transplant',
 'rx_mail_mbr_resp_pmpm_cost_t_9-6-3m_b4',
 'auth_3mth_acute_sns',
 'auth_3mth_post_acute_vco',
 'auth_3mth_home',
 'rx_nonbh_net_paid_pmpm_cost_t_6-3-0m_b4',
 'auth_3mth_post_acute_ckd',
 'rx_gpi2_34_dist_gpi6_pmpm_ct',
 'rx_gpi2_33_pmpm_ct_0to3m_b4',
 'auth_3mth_dc_ltac',
 'cons_estinv30_rc',
 'rx_phar_cat_humana_pmpm_ct_t_9-6-3m_b4',
 'auth_3mth_acute_men',
 'auth_3mth_dc_snf',
 'cons_hhcomp',
 'bh_ip_snf_mbr_resp_pmpm_cost_6to9m_b4',
 'auth_3mth_acute_inj',
 'total_physician_office_visit_ct_pmpm_t_6-3-0m_b4',
 'mabh_seg',
 'auth_3mth_post_acute_res',
 'auth_3mth_bh_acute_men',
 'auth_3mth_acute_hdz',
 'hedis_dia_hba1c_ge9',
 'auth_3mth_post_acute_trm',
 'auth_3mth_hospice',
 'rx_gpi2_39_pmpm_cost_t_6-3-0m_b4',
 'atlas_vlfoodsec_13_15',
 'auth_3mth_dc_acute_rehab',
 'rx_generic_pmpm_cost_t_6-3-0m_b4',
 'auth_3mth_acute_ccs_154',
 'cons_rxmaint',
 'total_bh_copay_pmpm_cost_t_9-6-3m_b4',
 'rx_nonmaint_dist_gpi6_pmpm_ct_t_12-9-6m_b4',
 'rej_med_outpatient_visit_ct_pmpm_t_6-3-0m_b4',
 'cons_rxadhm',
 'auth_3mth_acute_mus',
 'rx_nonbh_pmpm_cost_t_9-6-3m_b4',
 'rx_days_since_last_script_0to3m_b4',
 'auth_3mth_post_acute_cir',
 'auth_3mth_post_acute_dia',
 'auth_3mth_post_er',
 'auth_3mth_dc_no_ref',
 'bh_ip_snf_mbr_resp_pmpm_cost_3to6m_b4',
 'auth_3mth_acute',
 'rx_branded_pmpm_ct_t_6-3-0m_b4',
 'atlas_farm_to_school13',
 'auth_3mth_acute_cer',
 'med_ambulance_coins_pmpm_cost_t_9-6-3m_b4',
 'auth_3mth_acute_gus',
 'rx_gpi4_6110_pmpm_ct',
 'cons_hxwearbl',
 'auth_3mth_ltac',
 'auth_3mth_acute_ckd',
 'bh_ip_snf_net_paid_pmpm_cost_6to9m_b4',
 'sex_cd',
 'days_since_last_clm_0to3m_b4',
 'atlas_perpov_1980_0711',
 'auth_3mth_post_acute_mus',
 'auth_3mth_non_er',
 'bh_ncal_ind',
 'auth_3mth_facility',
 'atlas_foodinsec_13_15',
 'auth_3mth_dc_left_ama',
 'race_cd',
 'bh_ip_snf_admit_days_pmpm_t_9-6-3m_b4',
 'auth_3mth_dc_other',
 'cons_stlnindx',
 'auth_3mth_acute_skn',
 'total_allowed_pmpm_cost_t_9-6-3m_b4',
 'auth_3mth_rehab',
 'bh_urgent_care_copay_pmpm_cost_t_12-9-6m_b4',
 'auth_3mth_dc_custodial',
 'auth_3mth_snf_direct',
 'auth_3mth_acute_ccs_042',
 'bh_ip_snf_net_paid_pmpm_cost_9to12m_b4',
 'bh_ip_snf_net_paid_pmpm_cost_3to6m_b4',
 'rx_maint_pmpm_cost_t_12-9-6m_b4',
 'auth_3mth_post_acute_rsk',
 'rev_cms_ansth_pmpm_ct',
 'cons_cwht']

imp_features= ['cons_n2pwh',
 'days_since_last_clm_0to3m_b4',
 'cons_cwht',
 'rx_days_since_last_script_0to3m_b4',
 'cons_n2pmr',
 'est_age',
 'cons_n2phi',
 'atlas_foodinsec_13_15',
 'atlas_foodinsec_child_03_11',
 'atlas_vlfoodsec_13_15',
 'atlas_ghveg_farms12',
 'rx_gpi2_17_pmpm_cost_t_12-9-6m_b4',
 'cms_tot_partd_payment_amt',
 'race_cd',
 'src_div_id',
 'cms_risk_adjustment_factor_a_amt',
 'atlas_csa12',
 'rx_overall_mbr_resp_pmpm_cost_t_6-3-0m_b4',
 'rx_bh_pmpm_ct_0to3m_b4',
 'mabh_seg',
 'hum_region',
 'cons_estinv30_rc',
 'rx_generic_pmpm_ct_0to3m_b4',
 'credit_hh_bankcard_severederog',
 'rx_overall_gpi_pmpm_ct_0to3m_b4',
 'cnt_cp_webstatement_pmpm_ct',
 'credit_hh_bankcardcredit_60dpd',
 'rx_gpi2_39_pmpm_cost_t_6-3-0m_b4',
 'credit_hh_nonmtgcredit_60dpd',
 'rx_mail_net_paid_pmpm_cost_t_6-3-0m_b4',
 'cons_nwperadult',
 'cons_cgqs',
 'cons_rxadhm',
 'atlas_totalocchu',
 'cons_hhcomp',
 'rx_nonbh_mbr_resp_pmpm_cost',
 'lab_albumin_loinc_pmpm_ct',
 'rx_phar_cat_cvs_pmpm_ct_t_9-6-3m_b4',
 'rx_tier_2_pmpm_ct',
 'credit_bal_consumerfinance',
 'rx_maint_net_paid_pmpm_cost_t_12-9-6m_b4',
 'rx_generic_pmpm_cost',
 'rx_nonotc_pmpm_cost_t_6-3-0m_b4',
 'cms_orig_reas_entitle_cd',
 'atlas_recfac14',
 'cons_ltmedicr',
 'rx_overall_mbr_resp_pmpm_cost_0to3m_b4',
 'ccsp_227_pct',
 'cons_hxmioc',
 'rx_tier_1_pmpm_ct_0to3m_b4',
 'rx_generic_mbr_resp_pmpm_cost',
 'cons_stlnindx',
 'rx_tier_2_pmpm_ct_3to6m_b4',
 'mcc_chf_pmpm_ct_t_9-6-3m_b4',
 'rx_maint_pmpm_ct_9to12m_b4',
 'total_allowed_pmpm_cost_t_9-6-3m_b4',
 'rx_overall_mbr_resp_pmpm_cost',
 'cms_partd_ra_factor_amt',
 'rx_hum_16_pmpm_ct',
 'rx_generic_pmpm_cost_t_6-3-0m_b4',
 'atlas_pc_ffrsales12',
 'credit_bal_mtgcredit_new',
 'rx_mail_mbr_resp_pmpm_cost_t_9-6-3m_b4',
 'rx_overall_dist_gpi6_pmpm_ct_t_6-3-0m_b4',
 'rx_maint_pmpm_cost_t_6-3-0m_b4',
 'sex_cd',
 'lab_dist_loinc_pmpm_ct',
 'phy_em_px_pct',
 'rx_overall_gpi_pmpm_ct_t_12-9-6m_b4',
 'rx_nonbh_mbr_resp_pmpm_cost_6to9m_b4',
 'atlas_pc_dirsales12',
 'rx_maint_pmpm_cost_t_12-9-6m_b4',
 'rx_maint_net_paid_pmpm_cost_t_9-6-3m_b4',
 'rx_nonmaint_mbr_resp_pmpm_cost',
 'atlas_pct_snap16',
 'rx_generic_mbr_resp_pmpm_cost_0to3m_b4',
 'rx_generic_pmpm_cost_6to9m_b4',
 'credit_bal_nonmtgcredit_60dpd',
 'atlas_hipov_1115',
 'rx_maint_pmpm_ct_t_6-3-0m_b4',
 'cons_rxadhs',
 'med_outpatient_mbr_resp_pmpm_cost_t_9-6-3m_b4',
 'atlas_pct_sbp15',
 'atlas_pct_cacfp15',
 'total_med_net_paid_pmpm_cost_t_6-3-0m_b4',
 'rx_nonmaint_mbr_resp_pmpm_cost_9to12m_b4',
 'rx_overall_gpi_pmpm_ct_t_6-3-0m_b4',
 'atlas_pct_wic15',
 'lang_spoken_cd',
 'cci_dia_m_pmpm_ct',
 'atlas_pct_loclsale12',
 'met_obe_diag_pct',
 'rwjf_mv_deaths_rate',
 'atlas_pct_laccess_nhasian15',
 'rx_generic_dist_gpi6_pmpm_ct_t_9-6-3m_b4',
 'cons_rxmaint',
 'atlas_pct_reduced_lunch14',
 'cons_chva',
 'rx_nonotc_dist_gpi6_pmpm_ct',
 'atlas_netmigrationrate1016',
 'rx_nonbh_pmpm_cost_t_9-6-3m_b4',
 'cons_chmi',
 'rx_nonbh_pmpm_ct_t_9-6-3m_b4',
 'rx_phar_cat_humana_pmpm_ct_t_9-6-3m_b4',
 'rx_nonbh_pmpm_ct_0to3m_b4',
 'rx_nonbh_net_paid_pmpm_cost_t_6-3-0m_b4',
 'atlas_pct_fmrkt_otherfood16',
 'rx_gpi2_56_dist_gpi6_pmpm_ct_3to6m_b4',
 'atlas_pct_fmrkt_sfmnp16',
 'atlas_pct_nslp15',
 'auth_3mth_dc_home',
 'rej_med_outpatient_visit_ct_pmpm_t_6-3-0m_b4',
 'atlas_pct_laccess_hhnv15',
 'atlas_pct_laccess_lowi15',
 'atlas_net_international_migration_rate',
 'atlas_retirement_destination_2015_upda',
 'rx_days_since_last_script',
 'total_physician_office_mbr_resp_pmpm_cost_t_9-6-3m_b4',
 'rx_gpi2_49_pmpm_cost_0to3m_b4',
 'rx_gpi2_72_pmpm_ct_6to9m_b4',
 'atlas_pc_fsrsales12',
 'atlas_pct_sfsp15',
 'rx_nonmaint_dist_gpi6_pmpm_ct_t_12-9-6m_b4',
 'rx_gpi2_34_dist_gpi6_pmpm_ct',
 'auth_3mth_acute',
 'rx_nonmail_dist_gpi6_pmpm_ct_t_9-6-3m_b4',
 'atlas_orchard_farms12',
 'zip_cd',
 'credit_hh_1stmtgcredit',
 'atlas_pct_diabetes_adults13',
 'med_outpatient_visit_ct_pmpm_t_12-9-6m_b4',
 'atlas_pct_laccess_white15',
 'atlas_slhouse12',
 'rx_branded_pmpm_ct_t_6-3-0m_b4',
 'auth_3mth_home',
 'total_physician_office_net_paid_pmpm_cost_t_9-6-3m_b4',
 'atlas_pct_laccess_black15',
 'atlas_pct_fmrkt_anmlprod16',
 'oontwk_mbr_resp_pmpm_cost_t_6-3-0m_b4',
 'atlas_pct_laccess_nhna15',
 'cons_hxwearbl',
 'rx_mail_mbr_resp_pmpm_cost_0to3m_b4',
 'atlas_percapitainc',
 'med_physician_office_allowed_pmpm_cost_t_9-6-3m_b4',
 'atlas_pct_laccess_pop15',
 'atlas_pct_fmrkt_frveg16',
 'atlas_foodhub16',
 'rx_nonbh_net_paid_pmpm_cost',
 'atlas_pct_fmrkt_credit16',
 'total_physician_office_visit_ct_pmpm_t_6-3-0m_b4',
 'rej_days_since_last_clm',
 'rx_maint_mbr_resp_pmpm_cost_6to9m_b4',
 'atlas_agritrsm_rct12',
 'rx_nonmaint_pmpm_ct',
 'atlas_pc_wic_redemp12',
 'atlas_pct_free_lunch14',
 'rwjf_air_pollute_density',
 'atlas_redemp_snaps16',
 'credit_bal_bankcard_severederog',
 'auth_3mth_dc_snf',
 'atlas_orchard_acrespth12',
 'rx_gpi4_6110_pmpm_ct',
 'credit_bal_autobank',
 'atlas_type_2015_update',
 'rev_cms_ansth_pmpm_ct',
 'mcc_end_pct',
 'atlas_deep_pov_children',
 'rwjf_income_inequ_ratio',
 'ccsp_236_pct',
 'rwjf_social_associate_rate',
 'total_med_allowed_pmpm_cost_9to12m_b4',
 'atlas_pct_laccess_multir15',
 'credit_bal_autofinance_new',
 'atlas_medhhinc',
 'atlas_pct_obese_adults13',
 'total_outpatient_mbr_resp_pmpm_cost_6to9m_b4',
 'total_bh_copay_pmpm_cost_t_9-6-3m_b4',
 'rwjf_resident_seg_black_inx',
 'rx_gpi2_90_dist_gpi6_pmpm_ct_9to12m_b4',
 'atlas_hiamenity',
 'rx_overall_net_paid_pmpm_cost_6to9m_b4',
 'atlas_naturalchangerate1016',
 'atlas_pct_laccess_snap15',
 'rx_days_since_last_script_6to9m_b4',
 'rx_hum_28_pmpm_cost',
 'atlas_pct_loclfarm12',
 'atlas_dirsales_farms12',
 'atlas_freshveg_farms12',
 'atlas_ownhomepct',
 'cons_mobplus',
 'atlas_type_2015_mining_no',
 'auth_3mth_acute_mean_los',
 'atlas_berry_farms12',
 'atlas_pct_laccess_seniors15',
 'atlas_veg_farms12',
 'atlas_hh65plusalonepct',
 'rx_bh_mbr_resp_pmpm_cost_9to12m_b4',
 'atlas_povertyallagespct',
 'atlas_pct_fmrkt_baked16',
 'total_physician_office_copay_pmpm_cost',
 'atlas_pc_snapben15',
 'rx_gpi2_62_pmpm_cost_t_9-6-3m_b4',
 'bh_ip_snf_mbr_resp_pmpm_cost_3to6m_b4',
 'atlas_perpov_1980_0711',
 'rx_gpi2_33_pmpm_ct_0to3m_b4',
 'atlas_veg_acrespth12',
 'rx_branded_mbr_resp_pmpm_cost',
 'atlas_pct_laccess_hisp15',
 'bh_urgent_care_copay_pmpm_cost_t_12-9-6m_b4',
 'atlas_berry_acrespth12',
 'bh_ip_snf_mbr_resp_pmpm_cost_9to12m_b4',
 'atlas_ghveg_sqftpth12',
 'atlas_pct_fmrkt_snap16',
 'auth_3mth_facility',
 'atlas_deep_pov_all',
 'total_physician_office_net_paid_pmpm_cost_9to12m_b4',
 'total_ip_maternity_net_paid_pmpm_cost_t_12-9-6m_b4',
 'med_physician_office_ds_clm_6to9m_b4',
 'atlas_farm_to_school13',
 'bh_ip_snf_net_paid_pmpm_cost_9to12m_b4',
 'bh_physician_office_copay_pmpm_cost_6to9m_b4',
 'rx_gpi2_01_pmpm_cost_0to3m_b4',
 'atlas_persistentchildpoverty_1980_2011',
 'atlas_pct_laccess_child15',
 'atlas_povertyunder18pct',
 'atlas_pct_fmrkt_wiccash16',
 'rx_gpi2_72_pmpm_cost_6to9m_b4',
 'total_outpatient_allowed_pmpm_cost_6to9m_b4',
 'atlas_pct_fmrkt_wic16',
 'atlas_low_education_2015_update',
 'bh_ip_snf_net_paid_pmpm_cost_3to6m_b4',
 'atlas_type_2015_recreation_no',
 'auth_3mth_post_acute_mean_los',
 'bh_ncdm_pct',
 'auth_3mth_post_acute',
 'auth_3mth_acute_ccs_227',
 'auth_3mth_acute_ccs_153',
 'auth_3mth_acute_ccs_172',
 'auth_3mth_acute_ccs_086',
 'auth_3mth_acute_ccs_094',
 'auth_3mth_acute_ccs_154',
 'auth_3mth_post_acute_ner',
 'rx_gpi2_02_pmpm_cost',
 'auth_3mth_acute_ccs_067',
 'auth_3mth_acute_ccs_048',
 'auth_3mth_acute_ccs_044',
 'auth_3mth_acute_ccs_043',
 'auth_3mth_acute_ccs_042',
 'auth_3mth_acute_ccs_030',
 'auth_3mth_acute_can']
target = "covid_vaccination"


# reg_cols = [i for i in reg_cols if i in imp_features]
# cat_cols = [i for i in cat_cols if i in imp_features]

print(len(cat_cols), len(reg_cols))
student_id = 2000728661

id = "ID"
reg_scalar = MinMaxScaler()
reg_f, cat_f = fdf = SelectKBest(f_regression, k=50), SelectKBest(f_classif, k=70)
def scale_df(dataframe, train=True):
    
    if train: dataframe[reg_cols] = reg_scalar.fit_transform(dataframe[reg_cols])
    else: dataframe[reg_cols] = reg_scalar.transform(dataframe[reg_cols])
    return dataframe

def get_reduced_features(dataframe, train=True):
    rcat, rreg = [], []
    if train: 
        rcat = cat_f.fit(dataframe[cat_cols], dataframe[target]).get_support(indices=True)
        rreg = reg_f.fit(dataframe[reg_cols], dataframe[target]).get_support(indices=True)
    else:
        rcat = cat_f.get_support(indices=True)
        rreg = reg_f.get_support(indices=True)
    rcat = [i for idx, i in enumerate(dataframe[cat_cols].columns) if idx in rcat]
    rreg = [i for idx, i in enumerate(dataframe[reg_cols].columns) if idx in rreg]
    return rcat, rreg

def sc_df(dataframe):
    
    for col in dataframe.columns:
        if dataframe[col].dtype == np.int64:
            dataframe[col] = dataframe[col].astype(np.int32)

        if dataframe[col].dtype == np.float64:
            dataframe[col] = dataframe[col].astype(np.float32)
    return dataframe
df = pd.read_csv('dataset/transformed_dataset.csv')

# cat_cols, reg_cols = get_reduced_features(df)


df = df[cat_cols + reg_cols + [target]]
df=scale_df(df)
df = sc_df(df)
df.head()

191 139


,bh_ncdm_ind,auth_3mth_post_acute_inf,rx_maint_net_paid_pmpm_cost_t_9-6-3m_b4,ccsp_065_pmpm_ct,auth_3mth_acute_vco,rx_gpi2_72_pmpm_ct_6to9m_b4,auth_3mth_post_acute_men,rej_total_physician_office_visit_ct_pmpm_0to3m_b4,total_physician_office_net_paid_pmpm_cost_t_9-6-3m_b4,bh_ip_snf_net_paid_pmpm_cost_0to3m_b4,mcc_ano_pmpm_ct_t_9-6-3m_b4,atlas_type_2015_update,atlas_retirement_destination_2015_upda,auth_3mth_post_acute_sns,atlas_hiamenity,cons_ltmedicr,auth_3mth_acute_ccs_086,total_physician_office_mbr_resp_pmpm_cost_t_9-6-3m_b4,auth_3mth_acute_cir,atlas_csa12,total_med_net_paid_pmpm_cost_t_6-3-0m_b4,cons_n2pwh,auth_3mth_snf_post_hsp,auth_3mth_post_acute_inj,med_outpatient_mbr_resp_pmpm_cost_t_9-6-3m_b4,rx_gpi2_56_dist_gpi6_pmpm_ct_3to6m_b4,atlas_low_employment_2015_update,auth_3mth_acute_inf,lab_albumin_loinc_pmpm_ct,rx_gpi2_17_pmpm_cost_t_12-9-6m_b4,cons_rxadhs,cons_mobplus,atlas_foodinsec_child_03_11,lang_spoken_cd,bh_ip_snf_mbr_resp_pmpm_cost_9to12m_b4,auth_3mth_post_acute_gus,auth_3mth_acute_cad,rx_maint_pmpm_ct_t_6-3-0m_b4,auth_3mth_acute_ccs_044,cons_hxmioc,med_outpatient_visit_ct_pmpm_t_12-9-6m_b4,med_physician_office_allowed_pmpm_cost_t_9-6-3m_b4,auth_3mth_acute_res,auth_3mth_acute_chf,auth_3mth_acute_ccs_030,auth_3mth_dc_hospice,auth_3mth_acute_neo,atlas_type_2015_recreation_no,hum_region,atlas_ghveg_farms12,rx_maint_net_paid_pmpm_cost_t_12-9-6m_b4,auth_3mth_acute_ccs_048,rx_overall_gpi_pmpm_ct_t_6-3-0m_b4,rx_overall_gpi_pmpm_ct_t_12-9-6m_b4,rx_nonbh_pmpm_ct_t_9-6-3m_b4,mcc_chf_pmpm_ct_t_9-6-3m_b4,auth_3mth_post_acute_chf,auth_3mth_psychic,rx_nonotc_pmpm_cost_t_6-3-0m_b4,auth_3mth_acute_end,atlas_low_education_2015_update,src_div_id,auth_3mth_bh_acute,auth_3mth_acute_ccs_067,atlas_type_2015_mining_no,cons_n2pmr,rx_mail_net_paid_pmpm_cost_t_6-3-0m_b4,rej_med_er_net_paid_pmpm_cost_t_9-6-3m_b4,med_outpatient_deduct_pmpm_cost_t_9-6-3m_b4,rej_med_ip_snf_coins_pmpm_cost_t_9-6-3m_b4,rx_generic_dist_gpi6_pmpm_ct_t_9-6-3m_b4,auth_3mth_dc_home,auth_3mth_acute_bld,auth_3mth_acute_ner,oontwk_mbr_resp_pmpm_cost_t_6-3-0m_b4,rx_gpi2_90_dist_gpi6_pmpm_ct_9to12m_b4,atlas_foodhub16,rx_maint_pmpm_cost_t_6-3-0m_b4,auth_3mth_post_acute_ben,est_age,auth_3mth_post_acute_cer,auth_3mth_acute_ccs_153,auth_3mth_acute_dig,total_ip_maternity_net_paid_pmpm_cost_t_12-9-6m_b4,auth_3mth_post_acute_cad,rx_bh_pmpm_ct_0to3m_b4,rx_nonmail_dist_gpi6_pmpm_ct_t_9-6-3m_b4,atlas_persistentchildpoverty_1980_2011,atlas_slhouse12,atlas_population_loss_2015_update,auth_3mth_acute_ccs_094,auth_3mth_post_acute_ner,auth_3mth_acute_ccs_227,rx_overall_dist_gpi6_pmpm_ct_t_6-3-0m_b4,auth_3mth_acute_trm,auth_3mth_post_acute,auth_3mth_acute_dia,auth_3mth_acute_ccs_043,rx_overall_mbr_resp_pmpm_cost_t_6-3-0m_b4,cms_orig_reas_entitle_cd,auth_3mth_post_acute_end,auth_3mth_acute_can,auth_3mth_acute_ccs_172,auth_3mth_dc_home_health,atlas_hipov_1115,rx_phar_cat_cvs_pmpm_ct_t_9-6-3m_b4,rx_gpi2_62_pmpm_cost_t_9-6-3m_b4,cons_n2phi,auth_3mth_post_acute_hdz,auth_3mth_bh_acute_mean_los,auth_3mth_post_acute_dig,auth_3mth_transplant,rx_mail_mbr_resp_pmpm_cost_t_9-6-3m_b4,auth_3mth_acute_sns,auth_3mth_post_acute_vco,auth_3mth_home,rx_nonbh_net_paid_pmpm_cost_t_6-3-0m_b4,auth_3mth_post_acute_ckd,rx_gpi2_34_dist_gpi6_pmpm_ct,rx_gpi2_33_pmpm_ct_0to3m_b4,auth_3mth_dc_ltac,cons_estinv30_rc,rx_phar_cat_humana_pmpm_ct_t_9-6-3m_b4,auth_3mth_acute_men,auth_3mth_dc_snf,cons_hhcomp,bh_ip_snf_mbr_resp_pmpm_cost_6to9m_b4,auth_3mth_acute_inj,total_physician_office_visit_ct_pmpm_t_6-3-0m_b4,mabh_seg,auth_3mth_post_acute_res,auth_3mth_bh_acute_men,auth_3mth_acute_hdz,hedis_dia_hba1c_ge9,auth_3mth_post_acute_trm,auth_3mth_hospice,rx_gpi2_39_pmpm_cost_t_6-3-0m_b4,atlas_vlfoodsec_13_15,auth_3mth_dc_acute_rehab,rx_generic_pmpm_cost_t_6-3-0m_b4,auth_3mth_acute_ccs_154,cons_rxmaint,total_bh_copay_pmpm_cost_t_9-6-3m_b4,rx_nonmaint_dist_gpi6_pmpm_ct_t_12-9-6m_b4,rej_med_outpatient_visit_ct_pmpm_t_6-3-0m_b4,cons_rxadhm,auth_3mth_acute_mus,rx_nonbh_pmpm_cost_t_9-6-3m_b4,rx_days_since_last_script_0to3m_b4,auth_3mth

In [2]:


# X_T, y_T = df[reg_cols + cat_cols], df[target]
# X_T, X_t, y_T, y_t = train_test_split(X, y, test_size=.2, random_state=student_id, shuffle=True, stratify=y)


In [3]:
exp1 = setup(data=df[reg_cols + cat_cols + [target]], target=target, normalize=False, categorical_features=cat_cols, 
      data_split_stratify=True, 
                  ignore_low_variance = True, n_jobs=1,preprocess=False,
                  remove_multicollinearity = True, multicollinearity_threshold = 0.95, numeric_features=reg_cols, silent=True)

,Description,Value
0,session_id,4927
1,Target,covid_vaccination
2,Target Type,Binary
3,Label Encoded,None
4,Original Data,"(974842, 331)"
5,Missing Values,False
6,Numeric Features,139
7,Categorical Features,191
8,Transformed Train Set,"(682389, 330)"
9,Transformed Test Set,"(292453, 330)"


In [4]:
compare_models(cross_validation=False, sort='AUC', 
               include=['dt', 'rf', 'et', 'ada', 'lightgbm', 'catboost', 'gbc'])

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
catboost,CatBoost Classifier,0.8260,0.6808,0.0107,0.4784,0.0209,0.0134,0.0502,431.9600
gbc,Gradient Boosting Classifier,0.8262,0.6765,0.0014,0.5512,0.0027,0.0019,0.0208,1532.6000
ada,Ada Boost Classifier,0.8260,0.6678,0.0023,0.4260,0.0046,0.0027,0.0205,734.4300
et,Extra Trees Classifier,0.8262,0.6441,0.0003,0.4848,0.0006,0.0004,0.0087,252.5300
rf,Random Forest Classifier,0.8262,0.6434,0.0008,0.5125,0.0016,0.0011,0.0148,466.4900
dt,Decision Tree Classifier,0.7151,0.5248,0.2331,0.2109,0.2214,0.0476,0.0477,153.4500


In [5]:
catboost = create_model('catboost', fold=10)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.8263,0.6751,0.0114,0.5233,0.0223,0.0150,0.0568
1,0.8257,0.6795,0.0102,0.4368,0.0199,0.0121,0.0443
2,0.8259,0.6809,0.0108,0.4621,0.0211,0.0133,0.0486
3,0.8260,0.6828,0.0100,0.4760,0.0196,0.0126,0.0484
4,0.8258,0.6751,0.0089,0.4449,0.0174,0.0106,0.0421
5,0.8261,0.6745,0.0109,0.4905,0.0213,0.0138,0.0520
6,0.8260,0.6762,0.0094,0.4805,0.0184,0.0118,0.0472
7,0.8261,0.6756,0.0108,0.4830,0.0211,0.0136,0.0509
8,0.8260,0.6762,0.0102,0.4708,0.0200,0.0127,0.0482
9,0.8257,0.6772,0.0108,0.4399,0.0211,0.0128,0.0459


In [ ]:
bagged_catboost = ensemble_model(catboost)

IntProgress(value=0, description='Processing: ', max=6)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,13:30:37
Status,. . . . . . . . . . . . . . . . . .,Fitting 10 Folds
Estimator,. . . . . . . . . . . . . . . . . .,CatBoost Classifier


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC


In [ ]:
calibrated_cat = calibrate_model(bagged_catboost)

In [ ]:
tuned_cat = tune_model(caliberated_cat)